# Encoder-Decoder

In [1]:
from models.enc_and_dec import Encoder,Decoder
from models.simple_GAN import Discriminator
import option_parser
import torch

d:\anaconda3\envs\keraal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
parser = option_parser.get_parser()
args, unknown = parser.parse_known_args()
topo = (0, 0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 12, 13, 12, 15, 16, 17, 12, 19, 20, 21)
data = torch.rand((256, 64, 23, 3))
offset = torch.rand((256, 64, 23, 1))

In [4]:
#test encoder, decoder
enc = Encoder(args, topo)

z = enc(data, offset)
print(z.shape)
dec = Decoder(args,enc)
offset2 = torch.rand((256, 16,23,1))
reconstruct = dec(z, offset2)
print(reconstruct.shape)

torch.Size([256, 16, 23, 16])
torch.Size([256, 64, 23, 3])


In [5]:
dis = Discriminator(args,topo)
data = torch.rand((256, 64, 23, 3))
offset = torch.rand((256, 64, 23, 1))

new_input = torch.concat([data,offset],dim = -1)
res = dis(new_input)
print(res.shape)

torch.Size([256, 16, 1])


# Network

In [5]:
from torch.utils.data.dataloader import DataLoader
from data import create_dataset, get_character_names
from option_parser import get_args, try_mkdir
from models import create_model
import torch
args = get_args()
# print(args)
characters = get_character_names(args)
dataset = create_dataset(args, characters)
print(len(dataset))
try_mkdir(args.save_dir)
data_loader = DataLoader(dataset, batch_size = args.batch_size, num_workers=0)
model = create_model(args, dataset)
for step, motion in enumerate(data_loader):
    if step == 0:
        model.set_input(motion)
        model.optimize_parameters()

load from file ./datasets/train_set/npy/Mousey.npy
Window count: 523, total frame: 21582
load from file ./datasets/train_set/npy/Goblin.npy
Window count: 523, total frame: 21582
load from file ./datasets/train_set/npy/Mremireh.npy
Window count: 523, total frame: 21582
load from file ./datasets/train_set/npy/Vampire.npy
Window count: 523, total frame: 21582
523
[1, 3]
[0, 1]
[1, 3]
[1, 0]
[0, 3]
[0, 2]
[2, 1]
[2, 0]
[1, 3]
[1, 2]
[2, 1]
[3, 1]
[3, 1]
[0, 2]
[3, 1]
[3, 0]
[1, 0]
[3, 1]
[1, 2]
[1, 3]
[0, 1]
[2, 1]
[1, 0]
[0, 2]
[0, 1]
[0, 1]
[1, 3]
[2, 1]
[1, 2]
[3, 0]
[2, 3]
[2, 0]


d:\anaconda3\envs\keraal\lib\site-packages\torch\optim\adam.py:90: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


AttributeError: 'list' object has no attribute 'to'

In [6]:
a = torch.rand(23).unsqueeze(-1).unsqueeze(0)
print(a.shape)

torch.Size([1, 23, 1])


In [3]:
import torch.nn as nn
m = nn.Linear(3, 2)
input = torch.randn(128, 64, 23, 3)
output = m(input)
print(output.size())

torch.Size([128, 64, 23, 2])


In [1]:
from models.skeleton_gnn import Node2NodeLayer, Node2EdgeLayer, Edge2NodeLayer
import torch
def calculate_adj_matrix(topology):
    adj = torch.zeros(len(topology), len(topology))
    for i,j in enumerate(topology):
        if i == 0: #the root not in
            continue
        adj[j,i] = 1
    return adj

topo111 = [0, 0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 12, 13, 12, 15, 16, 17, 12, 19, 20, 21]
input_node = torch.rand((128, 32, 23, 3))
input_edge = torch.rand((128, 32, 23, 1))
batch_size = 128
frame = 32
length = 23
adj = calculate_adj_matrix(topo111)
adj_matrix = torch.cat(batch_size*frame*[adj]).reshape((batch_size, frame, length, length))
layer2 = Edge2NodeLayer(1,3,3)
output = layer2(input_node, input_edge, adj_matrix)


d:\anaconda3\envs\keraal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from torch.nn import UpsamplingBilinear2d, Upsample
import torch
input = torch.arange(1, 5, dtype=torch.float32).view(1, 1, 2, 2)

In [13]:
m = UpsamplingBilinear2d(scale_factor=1.679)
m2 = Upsample(scale_factor=(3,5), mode="bilinear")
# print(m(input))
print(m2(input).shape)

torch.Size([1, 1, 6, 10])


In [1]:
from models.skeleton_gnn import Node2EdgeLayer
import torch
m = Node2EdgeLayer(20,11,2)
x = torch.randn((1,64, 3, 3))
e = torch.randn((1,64, 3, 1))

adj_matrix = torch.Tensor([
    [
        [0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]
    ]
])

Parameter containing:
tensor([[-0.0493,  0.2842, -0.2260,  0.0230, -0.0785, -0.1820, -0.1898,  0.2175,
          0.2870, -0.0038,  0.2706],
        [ 0.2880,  0.0537, -0.0778,  0.2507,  0.0131, -0.0855,  0.0157, -0.0123,
          0.0627, -0.1443, -0.0664]], requires_grad=True)
tensor([[-0.4926,  2.8419, -2.2601,  0.2299, -0.7847, -1.8200, -1.8982,  2.1745,
          2.8703, -0.0376,  2.7064],
        [ 2.8797,  0.5373, -0.7784,  2.5070,  0.1313, -0.8552,  0.1572, -0.1228,
          0.6269, -1.4431, -0.6638]], grad_fn=<MulBackward0>)


d:\anaconda3\envs\keraal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
a = torch.rand((2,11))
print(a*10)

tensor([[6.5431, 4.2462, 2.5434, 2.2468, 4.4117, 1.4680, 6.9981, 2.7509, 2.5631,
         5.9814, 7.0614],
        [4.6820, 9.1086, 9.2908, 8.2059, 4.6102, 2.9461, 3.4283, 2.4934, 3.0841,
         5.9957, 8.7028]])
